In [12]:

import os
from LSTM import WeatherLSTM
from torch.utils.data import DataLoader as TorchDataLoader
from tqdm import tqdm

In [13]:
import torch
from torch.optim import lr_scheduler

import torch.optim as optim
import torch.nn as nn
from TemperatureDatasetLSTM import TemperatureDataset
from functools import partial
import os
import tempfile
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms

In [14]:
dataset = TemperatureDataset('data_temp_max', 'data_temp_min')

100%|██████████| 562/562 [00:08<00:00, 68.52it/s]


In [15]:
# Split the dataset into train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders for each set
batch_size = 32  # You can adjust this as needed

train_loader = TorchDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = TorchDataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = TorchDataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [16]:
activations_map = {
    'relu': torch.nn.ReLU,
    'tanh': torch.nn.Tanh,
    'sigmoid': torch.nn.Sigmoid,
    'leaky_relu': torch.nn.LeakyReLU,
    'elu': torch.nn.ELU,
    'gelu': torch.nn.GELU,
    'selu': torch.nn.SELU,
    'none': nn.Identity
}
# Prepare activations

# Build and train the best model on the full training set
sample_input, sample_target = dataset[0]
input_dim = sample_input.shape[-1]
output_dim = sample_target.shape[-1]
best_model = WeatherLSTM(input_size=input_dim, output_size=output_dim, sequence_length=7, hidden_size=64, num_layers=3, dropout=0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_model.to(device)
print(best_model)
optimizer = optim.Adam(best_model.parameters(), lr=0.001, weight_decay=1e-5)
criterion = nn.MSELoss()

# Use the full train_loader for training
num_epochs = 50
best_val_loss = float('inf')
for epoch in range(num_epochs):
    best_model.train()
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        inputs, targets = batch
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = best_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        val_loss = 0.0
        for inputs, targets in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}"):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = best_model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
        val_loss /= len(val_loader.dataset)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            # Save the best model
            torch.save(best_model.state_dict(), 'best_model_temp_lstm.pt')
torch.save(best_model.state_dict(), 'last_model_temp_lstm.pt')
# Evaluate on the test set
best_model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = best_model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item() * inputs.size(0)
test_loss /= len(test_loader.dataset)
print(f"Test loss: {test_loss}")

WeatherLSTM(
  (lstm): LSTM(5, 64, num_layers=3, batch_first=True)
  (bn_lstm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (fc1): Linear(in_features=64, out_features=25, bias=True)
  (bn_fc1): LayerNorm((25,), eps=1e-05, elementwise_affine=True)
  (relu): ReLU()
  (fc2): Linear(in_features=25, out_features=5, bias=True)
)


Training Epoch 1/50:   0%|          | 0/80522 [00:00<?, ?it/s]

Validation Epoch 50/50: 100%|██████████| 17255/17255 [00:32<00:00, 523.48it/s]


Test loss: 0.03473906731774523


In [18]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, f1_score, accuracy_score

# Collect all predictions and targets from the test set
sample_input, sample_target = dataset[0]
input_dim = sample_input.shape[-1]
output_dim = sample_target.shape[-1]
best_model = WeatherLSTM(input_size=input_dim, output_size=output_dim, sequence_length=7, hidden_size=64, num_layers=3, dropout=0)
best_model.load_state_dict(torch.load('last_model_temp_lstm.pt'))
best_model.to(device)
all_preds = []
all_targets = []
best_model.eval()
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = best_model(inputs)
        all_preds.append(outputs.cpu().numpy())
        all_targets.append(targets.cpu().numpy())
all_preds = np.concatenate(all_preds, axis=0)
all_targets = np.concatenate(all_targets, axis=0)
# Calculate regression metrics
mse = mean_squared_error(all_targets, all_preds)
mae = mean_absolute_error(all_targets, all_preds)
r2 = r2_score(all_targets, all_preds)

print(f"Mean Squared Error: {mse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"R2 Score: {r2:.4f}")



Mean Squared Error: 0.0347
Mean Absolute Error: 0.0888
R2 Score: 0.3653
